# Twitter: sentiments analysis

## Import packages

In [1]:
import requests
import tweepy
import pandas as pd
import numpy as np
import twitter
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
from nltk import tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
import datetime as dt
import ast
from datetime import datetime
from datetime import timezone 
import sqlite3


C:\Users\DIARRA née CISSE Bal\Anaconda3\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


## Tweets scrapping

In [58]:
def tweet_search(query, max_tweets,langue):
    ''' Function that takes in a search string 'query', the maximum
        number of tweets 'max_tweets', and the minimum (i.e., starting)
        tweet id. It returns a list of tweepy.models.Status objects. '''
    
    global liste
    api = twitter.Api(consumer_key='GsZD2aeSrKjX1mhpyLwQ3sk6D',
  consumer_secret='i6h6mcoKUQ4ReffNRnZWMO2cOvZJg61IugUj2kSjpGmalPdFG6',
  access_token_key='969881872124841984-gOZs4DfzAWD2XlxqWsOIam9hestJWlH',
  access_token_secret='eTBWSq7ju1Qap896DLQxZDZN2bBkpt1elmnKkCZ7KXnYJ')
 
    searched_tweets = []
    while len(searched_tweets) < max_tweets:
        remaining_tweets = max_tweets - len(searched_tweets)
        print("remaining_tweets:",remaining_tweets)
        try:
            new_tweets = api.GetSearch(query, count=100,lang=langue)
            print('found',len(new_tweets),'tweets')
            for tweet in new_tweets:
                liste.append({"id":tweet.id,"name":tweet.user.name,"tweet": tweet.text,"created at": tweet.created_at, "place":tweet.place})
                #print("id:",tweet.id,"name:",tweet.user.name,"tweet:",tweet.text,"langue:",tweet.lang,"created at:", tweet.created_at, "location:", tweet.location,"place:",tweet.place,"user_name:",tweet.user.location,"user_location:",tweet.user.location)
            if not new_tweets:
                print('no tweets found')
                break
            searched_tweets.extend(new_tweets)
        except twitter.error.TwitterError: 
            print('exception raised, waiting 15 minutes')
            print('(until:', dt.datetime.now()+dt.timedelta(minutes=15), ')')
            time.sleep(15*60)
            #break # stop the loop
    return liste

liste=[]
a=tweet_search(query="e",max_tweets=10000,langue="en")
df_tweets=pd.DataFrame(a)
#import tempfile
#tempfile.tempdir='C:/Temp/'
#with tempfile.NamedTemporaryFile() as temp:

#    df_tweets.to_csv(temp.name + '.csv')

remaining_tweets: 10000
found 100 tweets
remaining_tweets: 9900
found 100 tweets
remaining_tweets: 9800
found 100 tweets
remaining_tweets: 9700
found 100 tweets
remaining_tweets: 9600
found 100 tweets
remaining_tweets: 9500
found 100 tweets
remaining_tweets: 9400
found 100 tweets
remaining_tweets: 9300
found 100 tweets
remaining_tweets: 9200
found 100 tweets
remaining_tweets: 9100
found 100 tweets
remaining_tweets: 9000
found 100 tweets
remaining_tweets: 8900
found 100 tweets
remaining_tweets: 8800
found 100 tweets
remaining_tweets: 8700
found 100 tweets
remaining_tweets: 8600
found 100 tweets
remaining_tweets: 8500
found 100 tweets
remaining_tweets: 8400
found 100 tweets
remaining_tweets: 8300
found 100 tweets
remaining_tweets: 8200
found 100 tweets
remaining_tweets: 8100
found 100 tweets
remaining_tweets: 8000
found 100 tweets
remaining_tweets: 7900
found 100 tweets
remaining_tweets: 7800
found 100 tweets
remaining_tweets: 7700
found 100 tweets
remaining_tweets: 7600
found 100 tweets

In [ ]:
#tweets1=pd.read_csv("tweets1.csv",encoding = "ISO-8859-1") 
#tweets2=pd.read_csv("tweets2.csv",encoding = "ISO-8859-1",engine='python') 
#tweets3=pd.read_csv("tweets3.csv",encoding = "ISO-8859-1",low_memory=False)
#tweets4=pd.read_csv("tweets4.csv",encoding = "ISO-8859-1",engine='python')

## Tweets cleaning

In [61]:
def clean_tweets(liste_df):
    df_tweets=pd.concat(liste_df)
    # Drop tweets chere place is unknow
    df_tweets=df_tweets.loc[df_tweets.place.notnull()]
    # Drop duplicates
    df_tweets.drop_duplicates(subset='tweet',inplace=True)
    df_tweets.reset_index(drop=True, inplace=True)
    dateTime=[]
    df_tweets['id']=df_tweets['id'].astype('int64')
    # column country and country_code
    df_tweets['place']=df_tweets['place'].astype('str')
    df_tweets=df_tweets.join(df_tweets['place'].apply(ast.literal_eval).apply(pd.Series)['country'])
    df_tweets=df_tweets.join(df_tweets['place'].apply(ast.literal_eval).apply(pd.Series)['country_code'])
    
    # Date parsing
    for x in df_tweets["created at"]:
        dateTime.append(datetime.strptime(x, '%a %b %d %H:%M:%S %z %Y').replace(
            tzinfo=timezone.utc).astimezone(tz=None).strftime('%Y-%m-%d %H:%M:%S'))
    df_tweets['datetime']=dateTime
    # from country code of two letters to country code of tree letters
    country_code= pd.read_csv("country_codes.csv",encoding = "ISO-8859-1",sep=";",names=["Country","country_code","country_code3"])  
    for i in range(0,len(df_tweets)):
        if any(country_code['country_code']==df_tweets['country_code'].loc[i]):
            df_tweets['country_code'].loc[i]=country_code[country_code['country_code']==df_tweets['country_code'].loc[i]]['country_code3'].values[0]
    # Dropping useless columns
    if len(liste_df) ==1:
        df_tweets.drop(['created at'], axis=1, inplace=True)

    else:
        df_tweets.drop(['Unnamed: 0','created at','location','user_name','user_location','langue'], axis=1, inplace=True)
    return(df_tweets)

df_tweets=clean_tweets([df_tweets])

C:\Users\DIARRA née CISSE Bal\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


## Sentiments analysis

In [63]:
def tweet_analysis(df_tweets):
    
    print( "analyse des sentiments" )
    df_tweets['text_lem'] = [''.join([WordNetLemmatizer().lemmatize(re.sub('[^A-Za-z]', ' ', line)) 
                                  for line in lists]).strip() for lists in df_tweets['tweet']]       
    sid = SentimentIntensityAnalyzer()
    df_tweets['sentiment_compound_polarity']=df_tweets.text_lem.apply(lambda x:sid.polarity_scores(x)['compound'])
    df_tweets['sentiment_neutral']=df_tweets.text_lem.apply(lambda x:sid.polarity_scores(x)['neu'])
    df_tweets['sentiment_negative']=df_tweets.text_lem.apply(lambda x:sid.polarity_scores(x)['neg'])
    df_tweets['sentiment_pos']=df_tweets.text_lem.apply(lambda x:sid.polarity_scores(x)['pos'])
    df_tweets['sentiment_type']=''
    df_tweets.loc[df_tweets.sentiment_compound_polarity>0,'sentiment_type']='POSITIVE'
    df_tweets.loc[df_tweets.sentiment_compound_polarity==0,'sentiment_type']='NEUTRAL'
    df_tweets.loc[df_tweets.sentiment_compound_polarity<0,'sentiment_type']='NEGATIVE'
    df_tweets.drop(['sentiment_neutral','sentiment_negative','sentiment_pos',], axis=1, inplace=True)
    print( "analyse finie" )
    return df_tweets
df_tweets=tweet_analysis(df_tweets)

analyse des sentiments
analyse finie


## Gender prediction

In [64]:
def pred_genre(dataframe):
    print("prediction des genres")
    print("reading names.csv")
    genders = pd.read_csv("./names.csv", sep = ',', header=0, encoding='latin-1')
    sizes = np.zeros(len(genders))
    names = pd.Series("",range(len(genders)))
    for i, row in genders.iterrows():
        sizes[i] = len(row['Name'])
        names[i] = row['Name'].lower()
    genders['Length'] = sizes
    genders['Name'] = names
    genders.sort_values(['Length'],ascending=False,inplace=True)
    
    dico = {}
    for i, row in genders.iterrows():
        dico[row['Name']]=row['prob.gender']
        
    male_titles = ['m', 'mr']
    female_titles = ['mrs', 'ms', 'miss', 'lady', 'mme', 'mlle']
    
    def pred_genre_2( username, dico, male_titles, female_titles):
        for t in male_titles:
            if( username.startswith(t+".") or username.startswith(t+" ")):
                return 'Male'
        for t in female_titles:
            if( username.startswith(t+".") or username.startswith(t+" ")):
                return 'Female'
        l1 = username.split(" ")
        mots = []
        for i in l1:
            l2 = i.split("-")
            mots += l2
        for mot in mots:
            try:
                genre = dico[mot]
                if( genre != 'Unknown'):
                    return genre
            except KeyError:
                pass
        return 'Unknown'
        
    print("prediction des genres")
    genres_predits = pd.Series(len(dataframe))
    for i, row in dataframe.iterrows() :
         genres_predits[i] = pred_genre_2(row['name'].lower(), dico, male_titles, female_titles)
    
    dataframe['gender_predicted'] = genres_predits
    print("prediction finie")
    return dataframe

df_tweets = pred_genre(df_tweets)


prediction des genres
reading names.csv
prediction des genres
prediction finie


## To SQLite database

In [65]:
def df_toSQLite(df_tweets):
    conn = sqlite3.connect("projet inf\projet_inf\db.sqlite3")
    cur = conn.cursor()
    df_tweets.to_sql("sentiments_analysis_tweets", conn,if_exists='append', index= False)
    cur.close()
    conn.close()
    return(df_tweets)
df_tweets=df_toSQLite(df_tweets)